In [ ]:
import csv
import os
import re
import tensorflow as tf

from object_detection import evaluator
from object_detection.core import standard_fields
from object_detection.metrics import tf_example_parser
from object_detection.utils import config_util
from object_detection.utils import label_map_util

import numpy as np
# flags = tf.app.flags
# tf.logging.set_verbosity(tf.logging.INFO)

# flags.DEFINE_string('eval_dir', None, 'Directory to write eval summaries to.')
# flags.DEFINE_string('eval_config_path', None,
#                     'Path to an eval_pb2.EvalConfig config file.')
# flags.DEFINE_string('input_config_path', None,
#                     'Path to an eval_pb2.InputConfig config file.')

# FLAGS = flags.FLAGS

In [ ]:
def _generate_filenames(filenames):
  result = []
  for filename in filenames:
    result += _generate_sharded_filenames(filename)
  return result

In [ ]:
def _generate_sharded_filenames(filename):
  m = re.search(r'@(\d{1,})', filename)
  if m:
    num_shards = int(m.group(1))
    return [
        re.sub(r'@(\d{1,})', '-%.5d-of-%.5d' % (i, num_shards), filename)
        for i in range(num_shards)
    ]
  else:
    return [filename]

In [ ]:
  ## model 1, from v2, train_out_547_8GPU8batch_frozen, load all images, 90367
    required_flags = ['input_config_path', 'eval_config_path', 'eval_dir']
#   for flag_name in required_flags:
#     if not getattr(FLAGS, flag_name):
#       raise ValueError('Flag --{} is required'.format(flag_name))

  configs = config_util.get_configs_from_multiple_files(
      eval_input_config_path='/root/train_eval_metrics/train_input_config.pbtxt',
      eval_config_path='/root/train_eval_metrics/train_eval_config.pbtxt')

  eval_config = configs['eval_config']
  input_config = configs['eval_input_config']

In [ ]:
  # 547 per 300, oid_per300_ckpt476120
  required_flags = ['input_config_path', 'eval_config_path', 'eval_dir']
#   for flag_name in required_flags:
#     if not getattr(FLAGS, flag_name):
#       raise ValueError('Flag --{} is required'.format(flag_name))

  configs = config_util.get_configs_from_multiple_files(
      eval_input_config_path='/root/oid_per300_ckpt476120_eval_metrics/train_input_config.pbtxt',
      eval_config_path='/root/oid_per300_ckpt476120_eval_metrics/train_eval_config.pbtxt')

  eval_config = configs['eval_config']
  input_config = configs['eval_input_config']

In [ ]:
  if input_config.WhichOneof('input_reader') == 'tf_record_input_reader':
    input_paths = input_config.tf_record_input_reader.input_path

    label_map = label_map_util.load_labelmap(input_config.label_map_path)
    max_num_classes = max([item.id for item in label_map.item])
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes)

    object_detection_evaluators = evaluator.get_evaluators(
        eval_config, categories)
    # Support a single evaluator
    object_detection_evaluator = object_detection_evaluators[0]

    skipped_images = 0
    processed_images = 0

In [ ]:
    for input_path in _generate_filenames(input_paths):
      tf.logging.info('Processing file: {0}'.format(input_path))

      record_iterator = tf.python_io.tf_record_iterator(path=input_path)
      data_parser = tf_example_parser.TfExampleDetectionAndGTParser()

      for string_record in record_iterator:
        tf.logging.log_every_n(tf.logging.INFO, 'Processed %d images...', 1000,
                               processed_images)
        processed_images += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        decoded_dict = data_parser.parse(example)
        print(decoded_dict)

        if decoded_dict:
          object_detection_evaluator.add_single_ground_truth_image_info(
              decoded_dict[standard_fields.DetectionResultFields.key],
              decoded_dict)
          object_detection_evaluator.add_single_detected_image_info(
              decoded_dict[standard_fields.DetectionResultFields.key],
              decoded_dict)
        else:
          skipped_images += 1
          tf.logging.info('Skipped images: {0}'.format(skipped_images))
        break
      break

In [ ]:
standard_fields.DetectionResultFields.key 

In [ ]:
decoded_dict[standard_fields.DetectionResultFields.key] # image id

In [ ]:
object_detection_evaluator.evaluate()

In [ ]:
##add all grouth class

In [ ]:
    all_class_dict = {}
    for input_path in _generate_filenames(input_paths):
      tf.logging.info('Processing file: {0}'.format(input_path))

      record_iterator = tf.python_io.tf_record_iterator(path=input_path)
      data_parser = tf_example_parser.TfExampleDetectionAndGTParser()

      for idx, string_record in enumerate(record_iterator):
        tf.logging.log_every_n(tf.logging.INFO, 'Processed %d images...', 100,
                               processed_images)
        processed_images += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        decoded_dict = data_parser.parse(example)
        all_class_dict[idx]=decoded_dict['groundtruth_classes']
        if idx == 149 or idx == 255:
            print(decoded_dict)
            decoded_dict['groundtruth_group_of']=np.zeros_like(decoded_dict['groundtruth_group_of'])
#         print(decoded_dict)

        if decoded_dict:
          object_detection_evaluator.add_single_ground_truth_image_info(
              decoded_dict[standard_fields.DetectionResultFields.key],
              decoded_dict)
          object_detection_evaluator.add_single_detected_image_info(
              decoded_dict[standard_fields.DetectionResultFields.key],
              decoded_dict)



In [ ]:
all_class_dict

In [ ]:
object_detection_evaluator.evaluate()

In [ ]:
all_class_dict0 = all_class_dict

In [ ]:
noGroudTruth=[134, 142, 185, 227, 250, 252, 265, 267, 276, 286, 303, 320, 340, 343, 402, 434, 439, 444,
 492, 515, 543]
noGroudTruthSet = set()
for key, item in all_class_dict0.items():
#     print(key)
#     print(item)
    for class_idx in item:
        if class_idx in noGroudTruth:
            noGroudTruthSet.add(key)

In [ ]:
    
#     all_class_dict = {}
    for input_path in _generate_filenames(input_paths):
      tf.logging.info('Processing file: {0}'.format(input_path))

      record_iterator = tf.python_io.tf_record_iterator(path=input_path)
      data_parser = tf_example_parser.TfExampleDetectionAndGTParser()

      for idx, string_record in enumerate(record_iterator):
        tf.logging.log_every_n(tf.logging.INFO, 'Processed %d images...', 100,
                               processed_images)
        processed_images += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        decoded_dict = data_parser.parse(example)
#         all_class_dict[idx]=decoded_dict['groundtruth_classes']
        if idx in noGroudTruthSet:
#             print(decoded_dict)
            decoded_dict['groundtruth_group_of']=np.zeros_like(decoded_dict['groundtruth_group_of'])
#         print(decoded_dict)

        if decoded_dict:
          object_detection_evaluator.add_single_ground_truth_image_info(
              decoded_dict[standard_fields.DetectionResultFields.key],
              decoded_dict)
          object_detection_evaluator.add_single_detected_image_info(
              decoded_dict[standard_fields.DetectionResultFields.key],
              decoded_dict)




In [ ]:
object_detection_evaluator.evaluate()

In [ ]:
## check file with image

In [ ]:
    
#     all_class_dict = {}
    for input_path in _generate_filenames(input_paths):
      tf.logging.info('Processing file: {0}'.format(input_path))

      record_iterator = tf.python_io.tf_record_iterator(path=input_path)
      data_parser = tf_example_parser.TfExampleDetectionAndGTParser()

      for idx, string_record in enumerate(record_iterator):
        tf.logging.log_every_n(tf.logging.INFO, 'Processed %d images...', 100,
                               processed_images)
        processed_images += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        print(example)
        decoded_dict = data_parser.parse(example)
        
#         print(decoded_dict)
        break
      break
      
#         all_class_dict[idx]=decoded_dict['groundtruth_classes']
#         if idx in noGroudTruthSet:
#             print(decoded_dict)
#             decoded_dict['groundtruth_group_of']=np.zeros_like(decoded_dict['groundtruth_group_of'])
#         print(decoded_dict)

#         if decoded_dict:
#           object_detection_evaluator.add_single_ground_truth_image_info(
#               decoded_dict[standard_fields.DetectionResultFields.key],
#               decoded_dict)
#           object_detection_evaluator.add_single_detected_image_info(
#               decoded_dict[standard_fields.DetectionResultFields.key],
#               decoded_dict)



